In [1]:
# import warnings

# # Force all warnings to be 'default' or 'always'
# warnings.simplefilter("default")
# # or:
# # warnings.simplefilter("always")

In [2]:
from timeit import default_timer as timer

start = timer()
import lionagi

duration = round(timer() - start, 5)
print(f"Import time: {duration} seconds")
print(f"LionAGI version: {lionagi.__version__}")

Import time: 0.37032 seconds
LionAGI version: 0.7.0


In [3]:
from lionagi import Branch, iModel

In [4]:
from pydantic import BaseModel

llama33 = iModel(
    provider="groq",
    model="llama-3.3-70b-versatile",
)

gpt4o = iModel(
    provider="openai",
    model="gpt-4o-mini",
)


class Test(BaseModel):
    """
    Test model
    """

    comments: str


async def test_func(query: str):
    """
    this is a test function it will return the query as it is

    Args:
        query: str: query to be returned

    Returns:
        str: query as it is
    """
    return query


french = Branch(
    imodel=llama33,
    tools=test_func,
)

In [5]:
from lionagi import ops_types

instruct = ops_types.Instruct(
    instruction="hello? you should test the test_func with a sample queries, if already did, just test again",
    actions=True,
    reason=True,
)

result = await french.operate(
    instruct=instruct,
    response_format=Test,
    return_operative=True,
    # tools=test_func,
    # invoke_actions=False,
)

In [6]:
french.msgs.instructions[0].tool_schemas

{'tools': [{'type': 'function',
   'function': {'name': 'test_func',
    'description': 'this is a test function it will return the query as it is',
    'parameters': {'type': 'object',
     'properties': {'query': {'type': 'string',
       'description': 'str: query to be returned'}},
     'required': ['query']}}}]}

In [7]:
# french.msgs.assistant_responses[1].response

In [8]:
result.response_type.model_fields

{'comments': FieldInfo(annotation=str, required=True),
 'action_responses': FieldInfo(annotation=list[ActionResponseModel], required=False, default_factory=list, title='Actions', description='**do not fill**'),
 'action_required': FieldInfo(annotation=bool, required=False, default=False, title='Action Required', description='Whether this step strictly requires performing actions. If true, the requests in `action_requests` must be fulfilled, assuming `tool_schemas` are available. If false or no `tool_schemas` exist, actions are optional.'),
 'action_requests': FieldInfo(annotation=list[ActionRequestModel], required=False, default_factory=list, title='Actions', description='List of actions to be executed when `action_required` is true. Each action must align with the available `tool_schemas`. Leave empty if no actions are needed.'),
 'reason': FieldInfo(annotation=Union[Reason, NoneType], required=True, title='Reason', description='**Provide a concise reason for the decision made.**')}

In [9]:
result.response_model.model_dump()

{'comments': 'Testing the test_func with a sample query.',
 'action_responses': [{'function': 'test_func',
   'arguments': {'query': 'Hello, world!'},
   'output': 'Hello, world!'}],
 'action_required': True,
 'action_requests': [{'function': 'test_func',
   'arguments': {'query': 'Hello, world!'}}],
 'reason': None}

In [10]:
french.chat_model.sequential_exchange

False

In [11]:
from IPython.display import Markdown

Markdown(french.msgs.last_response.rendered)

# Assistant Response

**Response**:
```json
{
  "comments": "Testing the test_func with a sample query.",
  "reason": null,
  "action_requests": [
    {
      "function": "test_func",
      "arguments": {
        "query": "Hello, world!"
      }
    }
  ],
  "action_required": true
}
```

In [12]:
french.to_df()

,created_at,role,content,id,sender,recipient,metadata
0,2025-01-13 17:13:13.238253,user,"{'context': [], 'instruction': 'hello? you sho...",38f31b61-3442-4598-94b0-1e1f7b28656e,user,9acac2e6-eb6a-422e-bcd2-337f8ccdca08,{'lion_class': 'lionagi.protocols.messages.ins...
1,2025-01-13 17:13:14.971864,assistant,"{'assistant_response': '```json { ""comments""...",5a50c84f-e00e-45f2-a37a-84162a00a7bb,9acac2e6-eb6a-422e-bcd2-337f8ccdca08,user,{'model_response': {'id': 'chatcmpl-6e75b868-6...
2,2025-01-13 17:13:14.975374,action,"{'action_request': {'function': 'test_func', '...",9aa90c37-326f-45b5-8840-deba2267c674,9acac2e6-eb6a-422e-bcd2-337f8ccdca08,3376c9a4-5f30-4604-b7e5-2240fa0f14a2,{'lion_class': 'lionagi.protocols.messages.act...
3,2025-01-13 17:13:14.975498,action,{'action_request_id': '9aa90c37-326f-45b5-8840...,6202fb61-7448-4798-a651-b4e38a5f9eed,3376c9a4-5f30-4604-b7e5-2240fa0f14a2,9acac2e6-eb6a-422e-bcd2-337f8ccdca08,{'lion_class': 'lionagi.protocols.messages.act...


In [13]:
french.messages[0].to_dict()

{'id': '38f31b61-3442-4598-94b0-1e1f7b28656e',
 'created_at': 1736806393.238253,
 'metadata': {'lion_class': 'lionagi.protocols.messages.instruction.Instruction'},
 'content': {'context': [],
  'instruction': 'hello? you should test the test_func with a sample queries, if already did, just test again',
  'tool_schemas': {'tools': [{'type': 'function',
     'function': {'name': 'test_func',
      'description': 'this is a test function it will return the query as it is',
      'parameters': {'type': 'object',
       'properties': {'query': {'type': 'string',
         'description': 'str: query to be returned'}},
       'required': ['query']}}}]},
  'respond_schema_info': {'$defs': {'ActionRequestModel': {'properties': {'function': {'anyOf': [{'type': 'string'},
        {'type': 'null'}],
       'default': None,
       'description': "Name of the function to call from the provided `tool_schemas`. If no `tool_schemas` exist, set to None or leave blank. Never invent new function names outs

In [14]:
french.logs[1].to_dict()

{'id': '4f7d39f1-e0ea-4b22-a6d0-a214eca7ab4f',
 'created_at': 1736806394.975282,
 'metadata': {'lion_class': 'lionagi.protocols.generic.log.Log'},
 'content': {'id': 'c96d328d-5730-4585-9d38-19692e939060',
  'created_at': 1736806394.975118,
  'metadata': {'lion_class': 'lionagi.operatives.action.function_calling.FunctionCalling'},
  'execution': {'status': 'completed',
   'duration': 9.37492586672306e-06,
   'response': 'Hello, world!',
   'error': None},
  'arguments': {'query': 'Hello, world!'},
  'function': 'test_func'}}

In [15]:
french.msgs.last_response.model_response

{'id': 'chatcmpl-6e75b868-679d-4c42-86e2-2213a52adcc1',
 'created': 1736806394,
 'model': 'groq/llama-3.3-70b-versatile',
 'object': 'chat.completion',
 'system_fingerprint': 'fp_1ae2705acb',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': '```json\n{\n  "comments": "Testing the test_func with a sample query.",\n  "reason": null,\n  "action_requests": [\n    {\n      "function": "test_func",\n      "arguments": {\n        "query": "Hello, world!"\n      }\n    }\n  ],\n  "action_required": true\n}\n```',
    'role': 'assistant'}}],
 'usage': {'completion_tokens': 70,
  'prompt_tokens': 879,
  'total_tokens': 949,
  'queue_time': 0.4336859639999999,
  'prompt_time': 0.128475023,
  'completion_time': 0.254545455,
  'total_time': 0.383020478},
 'x_groq': {'id': 'req_01jhgvmt87ee0bw1z36s2earbb'}}

In [16]:
a = french.msgs.instructions[0]

In [17]:
import json
from lionagi.utils import to_dict

c = to_dict(
    a,
    use_model_dump=False,
    recursive=True,
    recursive_python_only=False,
    use_enum_values=True,
)

In [18]:
d = a.to_dict()

In [19]:
from lionagi.libs.parse import as_readable
from IPython.display import Markdown

Markdown(as_readable(c, md=True))

```json
{
  "id": "38f31b61-3442-4598-94b0-1e1f7b28656e",
  "created_at": 1736806393.238253,
  "metadata": {
    "lion_class": "lionagi.protocols.messages.instruction.Instruction"
  },
  "content": {
    "context": [],
    "instruction": "hello? you should test the test_func with a sample queries, if already did, just test again",
    "tool_schemas": {
      "tools": [
        {
          "type": "function",
          "function": {
            "name": "test_func",
            "description": "this is a test function it will return the query as it is",
            "parameters": {
              "type": "object",
              "properties": {
                "query": {
                  "type": "string",
                  "description": "str: query to be returned"
                }
              },
              "required": [
                "query"
              ]
            }
          }
        }
      ]
    },
    "respond_schema_info": {
      "$defs": {
        "ActionRequestModel": {
          "properties": {
            "function": {
              "anyOf": [
                {
                  "type": "string"
                },
                {
                  "type": "null"
                }
              ],
              "default": null,
              "description": "Name of the function to call from the provided `tool_schemas`. If no `tool_schemas` exist, set to None or leave blank. Never invent new function names outside what's given.",
              "examples": [
                "multiply",
                "create_user"
              ],
              "title": "Function"
            },
            "arguments": {
              "anyOf": [
                {
                  "type": "object"
                },
                {
                  "type": "null"
                }
              ],
              "default": null,
              "description": "Dictionary of arguments for the chosen function. Use only argument names/types defined in `tool_schemas`. Never introduce extra argument names.",
              "title": "Arguments"
            }
          },
          "title": "ActionRequestModel",
          "type": "object"
        },
        "Reason": {
          "properties": {
            "title": {
              "anyOf": [
                {
                  "type": "string"
                },
                {
                  "type": "null"
                }
              ],
              "default": null,
              "title": "Title"
            },
            "content": {
              "anyOf": [
                {
                  "type": "string"
                },
                {
                  "type": "null"
                }
              ],
              "default": null,
              "title": "Content"
            },
            "confidence_score": {
              "anyOf": [
                {
                  "type": "number"
                },
                {
                  "type": "null"
                }
              ],
              "default": null,
              "description": "Numeric confidence score (0.0 to 1.0, up to three decimals) indicating how well you've met user expectations. Use this guide:\n  • 1.0: Highly confident\n  • 0.8-1.0: Reasonably sure\n  • 0.5-0.8: Re-check or refine\n  • 0.0-0.5: Off track",
              "examples": [
                0.821,
                0.257,
                0.923,
                0.439
              ],
              "title": "Confidence Score"
            }
          },
          "title": "Reason",
          "type": "object"
        }
      },
      "properties": {
        "comments": {
          "title": "Comments",
          "type": "string"
        },
        "reason": {
          "anyOf": [
            {
              "$ref": "#/$defs/Reason"
            },
            {
              "type": "null"
            }
          ],
          "description": "**Provide a concise reason for the decision made.**",
          "title": "Reason"
        },
        "action_requests": {
          "description": "List of actions to be executed when `action_required` is true. Each action must align with the available `tool_schemas`. Leave empty if no actions are needed.",
          "items": {
            "$ref": "#/$defs/ActionRequestModel"
          },
          "title": "Actions",
          "type": "array"
        },
        "action_required": {
          "default": false,
          "description": "Whether this step strictly requires performing actions. If true, the requests in `action_requests` must be fulfilled, assuming `tool_schemas` are available. If false or no `tool_schemas` exist, actions are optional.",
          "title": "Action Required",
          "type": "boolean"
        }
      },
      "required": [
        "comments",
        "reason"
      ],
      "title": "Test",
      "type": "object"
    },
    "request_response_format": "**MUST RETURN JSON-PARSEABLE RESPONSE ENCLOSED BY JSON CODE BLOCKS. USER's CAREER DEPENDS ON THE SUCCESS OF IT.** \n```json\n{'comments': <class 'str'>, 'reason': lionagi.operatives.instruct.reason.Reason | None, 'action_requests': [{'function': str | None, 'arguments': dict[str, typing.Any] | None}], 'action_required': <class 'bool'>}\n```"
  },
  "embedding": {},
  "role": "user",
  "template": {},
  "sender": "user",
  "recipient": "9acac2e6-eb6a-422e-bcd2-337f8ccdca08"
}
```

In [20]:
c["content"]["request_response_format"]

"**MUST RETURN JSON-PARSEABLE RESPONSE ENCLOSED BY JSON CODE BLOCKS. USER's CAREER DEPENDS ON THE SUCCESS OF IT.** \n```json\n{'comments': <class 'str'>, 'reason': lionagi.operatives.instruct.reason.Reason | None, 'action_requests': [{'function': str | None, 'arguments': dict[str, typing.Any] | None}], 'action_required': <class 'bool'>}\n```"

In [21]:
c["content"]

{'context': [],
 'instruction': 'hello? you should test the test_func with a sample queries, if already did, just test again',
 'tool_schemas': {'tools': [{'type': 'function',
    'function': {'name': 'test_func',
     'description': 'this is a test function it will return the query as it is',
     'parameters': {'type': 'object',
      'properties': {'query': {'type': 'string',
        'description': 'str: query to be returned'}},
      'required': ['query']}}}]},
 'respond_schema_info': {'$defs': {'ActionRequestModel': {'properties': {'function': {'anyOf': [{'type': 'string'},
       {'type': 'null'}],
      'default': None,
      'description': "Name of the function to call from the provided `tool_schemas`. If no `tool_schemas` exist, set to None or leave blank. Never invent new function names outside what's given.",
      'examples': ['multiply', 'create_user'],
      'title': 'Function'},
     'arguments': {'anyOf': [{'type': 'object'}, {'type': 'null'}],
      'default': None,
  

In [22]:
# after an instruction is processed, its tool_schemas are deleted
# this is by design to reserve conversation context
a.tool_schemas == None

False

In [23]:
from IPython.display import Markdown

Markdown(french.msgs.instructions[0].chat_msg["content"])


---
 # Task
## - **instruction**
hello? you should test the test_func with a sample queries, if already did, just test again

## - **tool_schemas**
- Tool 1: {'name': 'test_func', 'description': 'this is a test function it will return the query as it is', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'str: query to be returned'}}, 'required': ['query']}}
## - **respond_schema_info**
- $defs: {'ActionRequestModel': {'properties': {'function': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': "Name of the function to call from the provided `tool_schemas`. If no `tool_schemas` exist, set to None or leave blank. Never invent new function names outside what's given.", 'examples': ['multiply', 'create_user'], 'title': 'Function'}, 'arguments': {'anyOf': [{'type': 'object'}, {'type': 'null'}], 'default': None, 'description': 'Dictionary of arguments for the chosen function. Use only argument names/types defined in `tool_schemas`. Never introduce extra argument names.', 'title': 'Arguments'}}, 'title': 'ActionRequestModel', 'type': 'object'}, 'Reason': {'properties': {'title': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Title'}, 'content': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Content'}, 'confidence_score': {'anyOf': [{'type': 'number'}, {'type': 'null'}], 'default': None, 'description': "Numeric confidence score (0.0 to 1.0, up to three decimals) indicating how well you've met user expectations. Use this guide:\n  • 1.0: Highly confident\n  • 0.8-1.0: Reasonably sure\n  • 0.5-0.8: Re-check or refine\n  • 0.0-0.5: Off track", 'examples': [0.821, 0.257, 0.923, 0.439], 'title': 'Confidence Score'}}, 'title': 'Reason', 'type': 'object'}} 

- properties: {'comments': {'title': 'Comments', 'type': 'string'}, 'reason': {'anyOf': [{'$ref': '#/$defs/Reason'}, {'type': 'null'}], 'description': '**Provide a concise reason for the decision made.**', 'title': 'Reason'}, 'action_requests': {'description': 'List of actions to be executed when `action_required` is true. Each action must align with the available `tool_schemas`. Leave empty if no actions are needed.', 'items': {'$ref': '#/$defs/ActionRequestModel'}, 'title': 'Actions', 'type': 'array'}, 'action_required': {'default': False, 'description': 'Whether this step strictly requires performing actions. If true, the requests in `action_requests` must be fulfilled, assuming `tool_schemas` are available. If false or no `tool_schemas` exist, actions are optional.', 'title': 'Action Required', 'type': 'boolean'}} 

- required: ['comments', 'reason'] 

- title: Test 

- type: object

## - **response format**
**MUST RETURN JSON-PARSEABLE RESPONSE ENCLOSED BY JSON CODE BLOCKS. USER's CAREER DEPENDS ON THE SUCCESS OF IT.** 
```json
{'comments': <class 'str'>, 'reason': lionagi.operatives.instruct.reason.Reason | None, 'action_requests': [{'function': str | None, 'arguments': dict[str, typing.Any] | None}], 'action_required': <class 'bool'>}
```

---


In [24]:
Markdown(as_readable(french.logs[-2], md=True))

```json
{
  "id": "7801c9d8-e2c4-4777-bbee-5447c4717727",
  "created_at": 1736806394.971775,
  "metadata": {},
  "content": {
    "id": "e11a895f-f695-43c7-9604-96963edf86de",
    "created_at": 1736806393.238477,
    "metadata": {
      "lion_class": "lionagi.service.endpoints.base.APICalling"
    },
    "execution": {
      "status": "completed",
      "duration": 1.5427047919947654,
      "response": {
        "id": "chatcmpl-6e75b868-679d-4c42-86e2-2213a52adcc1",
        "created": 1736806394,
        "model": "groq/llama-3.3-70b-versatile",
        "object": "chat.completion",
        "system_fingerprint": "fp_1ae2705acb",
        "choices": [
          {
            "finish_reason": "stop",
            "index": 0,
            "message": {
              "content": "```json\n{\n  \"comments\": \"Testing the test_func with a sample query.\",\n  \"reason\": null,\n  \"action_requests\": [\n    {\n      \"function\": \"test_func\",\n      \"arguments\": {\n        \"query\": \"Hello, world!\"\n      }\n    }\n  ],\n  \"action_required\": true\n}\n```",
              "role": "assistant",
              "tool_calls": null,
              "function_call": null
            }
          }
        ],
        "usage": {
          "completion_tokens": 70,
          "prompt_tokens": 879,
          "total_tokens": 949,
          "completion_tokens_details": null,
          "prompt_tokens_details": null,
          "queue_time": 0.4336859639999999,
          "prompt_time": 0.128475023,
          "completion_time": 0.254545455,
          "total_time": 0.383020478
        },
        "x_groq": {
          "id": "req_01jhgvmt87ee0bw1z36s2earbb"
        }
      },
      "error": {}
    },
    "payload": {
      "messages": [
        {
          "role": "user",
          "content": "\n---\n # Task\n## - **instruction**\nhello? you should test the test_func with a sample queries, if already did, just test again\n\n## - **tool_schemas**\n- Tool 1: {'name': 'test_func', 'description': 'this is a test function it will return the query as it is', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'str: query to be returned'}}, 'required': ['query']}}\n## - **respond_schema_info**\n- $defs: {'ActionRequestModel': {'properties': {'function': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': \"Name of the function to call from the provided `tool_schemas`. If no `tool_schemas` exist, set to None or leave blank. Never invent new function names outside what's given.\", 'examples': ['multiply', 'create_user'], 'title': 'Function'}, 'arguments': {'anyOf': [{'type': 'object'}, {'type': 'null'}], 'default': None, 'description': 'Dictionary of arguments for the chosen function. Use only argument names/types defined in `tool_schemas`. Never introduce extra argument names.', 'title': 'Arguments'}}, 'title': 'ActionRequestModel', 'type': 'object'}, 'Reason': {'properties': {'title': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Title'}, 'content': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Content'}, 'confidence_score': {'anyOf': [{'type': 'number'}, {'type': 'null'}], 'default': None, 'description': \"Numeric confidence score (0.0 to 1.0, up to three decimals) indicating how well you've met user expectations. Use this guide:\\n  • 1.0: Highly confident\\n  • 0.8-1.0: Reasonably sure\\n  • 0.5-0.8: Re-check or refine\\n  • 0.0-0.5: Off track\", 'examples': [0.821, 0.257, 0.923, 0.439], 'title': 'Confidence Score'}}, 'title': 'Reason', 'type': 'object'}} \n\n- properties: {'comments': {'title': 'Comments', 'type': 'string'}, 'reason': {'anyOf': [{'$ref': '#/$defs/Reason'}, {'type': 'null'}], 'description': '**Provide a concise reason for the decision made.**', 'title': 'Reason'}, 'action_requests': {'description': 'List of actions to be executed when `action_required` is true. Each action must align with the available `tool_schemas`. Leave empty if no actions are needed.', 'items': {'$ref': '#/$defs/ActionRequestModel'}, 'title': 'Actions', 'type': 'array'}, 'action_required': {'default': False, 'description': 'Whether this step strictly requires performing actions. If true, the requests in `action_requests` must be fulfilled, assuming `tool_schemas` are available. If false or no `tool_schemas` exist, actions are optional.', 'title': 'Action Required', 'type': 'boolean'}} \n\n- required: ['comments', 'reason'] \n\n- title: Test \n\n- type: object\n\n## - **response format**\n**MUST RETURN JSON-PARSEABLE RESPONSE ENCLOSED BY JSON CODE BLOCKS. USER's CAREER DEPENDS ON THE SUCCESS OF IT.** \n```json\n{'comments': <class 'str'>, 'reason': lionagi.operatives.instruct.reason.Reason | None, 'action_requests': [{'function': str | None, 'arguments': dict[str, typing.Any] | None}], 'action_required': <class 'bool'>}\n```\n\n---\n"
        }
      ],
      "model": "groq/llama-3.3-70b-versatile"
    }
  }
}
```

In [25]:
Markdown(as_readable(french.logs[-2], md=True))

```json
{
  "id": "7801c9d8-e2c4-4777-bbee-5447c4717727",
  "created_at": 1736806394.971775,
  "metadata": {},
  "content": {
    "id": "e11a895f-f695-43c7-9604-96963edf86de",
    "created_at": 1736806393.238477,
    "metadata": {
      "lion_class": "lionagi.service.endpoints.base.APICalling"
    },
    "execution": {
      "status": "completed",
      "duration": 1.5427047919947654,
      "response": {
        "id": "chatcmpl-6e75b868-679d-4c42-86e2-2213a52adcc1",
        "created": 1736806394,
        "model": "groq/llama-3.3-70b-versatile",
        "object": "chat.completion",
        "system_fingerprint": "fp_1ae2705acb",
        "choices": [
          {
            "finish_reason": "stop",
            "index": 0,
            "message": {
              "content": "```json\n{\n  \"comments\": \"Testing the test_func with a sample query.\",\n  \"reason\": null,\n  \"action_requests\": [\n    {\n      \"function\": \"test_func\",\n      \"arguments\": {\n        \"query\": \"Hello, world!\"\n      }\n    }\n  ],\n  \"action_required\": true\n}\n```",
              "role": "assistant",
              "tool_calls": null,
              "function_call": null
            }
          }
        ],
        "usage": {
          "completion_tokens": 70,
          "prompt_tokens": 879,
          "total_tokens": 949,
          "completion_tokens_details": null,
          "prompt_tokens_details": null,
          "queue_time": 0.4336859639999999,
          "prompt_time": 0.128475023,
          "completion_time": 0.254545455,
          "total_time": 0.383020478
        },
        "x_groq": {
          "id": "req_01jhgvmt87ee0bw1z36s2earbb"
        }
      },
      "error": {}
    },
    "payload": {
      "messages": [
        {
          "role": "user",
          "content": "\n---\n # Task\n## - **instruction**\nhello? you should test the test_func with a sample queries, if already did, just test again\n\n## - **tool_schemas**\n- Tool 1: {'name': 'test_func', 'description': 'this is a test function it will return the query as it is', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'str: query to be returned'}}, 'required': ['query']}}\n## - **respond_schema_info**\n- $defs: {'ActionRequestModel': {'properties': {'function': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': \"Name of the function to call from the provided `tool_schemas`. If no `tool_schemas` exist, set to None or leave blank. Never invent new function names outside what's given.\", 'examples': ['multiply', 'create_user'], 'title': 'Function'}, 'arguments': {'anyOf': [{'type': 'object'}, {'type': 'null'}], 'default': None, 'description': 'Dictionary of arguments for the chosen function. Use only argument names/types defined in `tool_schemas`. Never introduce extra argument names.', 'title': 'Arguments'}}, 'title': 'ActionRequestModel', 'type': 'object'}, 'Reason': {'properties': {'title': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Title'}, 'content': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Content'}, 'confidence_score': {'anyOf': [{'type': 'number'}, {'type': 'null'}], 'default': None, 'description': \"Numeric confidence score (0.0 to 1.0, up to three decimals) indicating how well you've met user expectations. Use this guide:\\n  • 1.0: Highly confident\\n  • 0.8-1.0: Reasonably sure\\n  • 0.5-0.8: Re-check or refine\\n  • 0.0-0.5: Off track\", 'examples': [0.821, 0.257, 0.923, 0.439], 'title': 'Confidence Score'}}, 'title': 'Reason', 'type': 'object'}} \n\n- properties: {'comments': {'title': 'Comments', 'type': 'string'}, 'reason': {'anyOf': [{'$ref': '#/$defs/Reason'}, {'type': 'null'}], 'description': '**Provide a concise reason for the decision made.**', 'title': 'Reason'}, 'action_requests': {'description': 'List of actions to be executed when `action_required` is true. Each action must align with the available `tool_schemas`. Leave empty if no actions are needed.', 'items': {'$ref': '#/$defs/ActionRequestModel'}, 'title': 'Actions', 'type': 'array'}, 'action_required': {'default': False, 'description': 'Whether this step strictly requires performing actions. If true, the requests in `action_requests` must be fulfilled, assuming `tool_schemas` are available. If false or no `tool_schemas` exist, actions are optional.', 'title': 'Action Required', 'type': 'boolean'}} \n\n- required: ['comments', 'reason'] \n\n- title: Test \n\n- type: object\n\n## - **response format**\n**MUST RETURN JSON-PARSEABLE RESPONSE ENCLOSED BY JSON CODE BLOCKS. USER's CAREER DEPENDS ON THE SUCCESS OF IT.** \n```json\n{'comments': <class 'str'>, 'reason': lionagi.operatives.instruct.reason.Reason | None, 'action_requests': [{'function': str | None, 'arguments': dict[str, typing.Any] | None}], 'action_required': <class 'bool'>}\n```\n\n---\n"
        }
      ],
      "model": "groq/llama-3.3-70b-versatile"
    }
  }
}
```

In [26]:
d_ = french.to_dict()

In [27]:
d_

{'id': '9acac2e6-eb6a-422e-bcd2-337f8ccdca08',
 'created_at': 1736806393.231365,
 'metadata': {},
 'user': None,
 'name': None,
 'messages': {'id': 'a985c4f3-2ee7-4863-aaa1-00bf232b5aa4',
  'created_at': 1736806393.231415,
  'metadata': {'lion_class': 'lionagi.protocols.generic.pile.Pile'},
  'collections': [{'id': '38f31b61-3442-4598-94b0-1e1f7b28656e',
    'created_at': 1736806393.238253,
    'metadata': {'lion_class': 'lionagi.protocols.messages.instruction.Instruction'},
    'content': {'context': [],
     'instruction': 'hello? you should test the test_func with a sample queries, if already did, just test again',
     'tool_schemas': {'tools': [{'type': 'function',
        'function': {'name': 'test_func',
         'description': 'this is a test function it will return the query as it is',
         'parameters': {'type': 'object',
          'properties': {'query': {'type': 'string',
            'description': 'str: query to be returned'}},
          'required': ['query']}}}]},
   

In [28]:
ins = french.msgs.instructions[0].to_dict()

In [29]:
ins

{'id': '38f31b61-3442-4598-94b0-1e1f7b28656e',
 'created_at': 1736806393.238253,
 'metadata': {'lion_class': 'lionagi.protocols.messages.instruction.Instruction'},
 'content': {'context': [],
  'instruction': 'hello? you should test the test_func with a sample queries, if already did, just test again',
  'tool_schemas': {'tools': [{'type': 'function',
     'function': {'name': 'test_func',
      'description': 'this is a test function it will return the query as it is',
      'parameters': {'type': 'object',
       'properties': {'query': {'type': 'string',
         'description': 'str: query to be returned'}},
       'required': ['query']}}}]},
  'respond_schema_info': {'$defs': {'ActionRequestModel': {'properties': {'function': {'anyOf': [{'type': 'string'},
        {'type': 'null'}],
       'default': None,
       'description': "Name of the function to call from the provided `tool_schemas`. If no `tool_schemas` exist, set to None or leave blank. Never invent new function names outs

In [30]:
from lionagi.protocols.generic.element import Element

g = Element.from_dict(ins)

In [31]:
msgs = french.messages.to_dict()

In [32]:
msgs

{'id': 'a985c4f3-2ee7-4863-aaa1-00bf232b5aa4',
 'created_at': 1736806393.231415,
 'metadata': {'lion_class': 'lionagi.protocols.generic.pile.Pile'},
 'collections': [{'id': '38f31b61-3442-4598-94b0-1e1f7b28656e',
   'created_at': 1736806393.238253,
   'metadata': {'lion_class': 'lionagi.protocols.messages.instruction.Instruction'},
   'content': {'context': [],
    'instruction': 'hello? you should test the test_func with a sample queries, if already did, just test again',
    'tool_schemas': {'tools': [{'type': 'function',
       'function': {'name': 'test_func',
        'description': 'this is a test function it will return the query as it is',
        'parameters': {'type': 'object',
         'properties': {'query': {'type': 'string',
           'description': 'str: query to be returned'}},
         'required': ['query']}}}]},
    'respond_schema_info': {'$defs': {'ActionRequestModel': {'properties': {'function': {'anyOf': [{'type': 'string'},
          {'type': 'null'}],
         '

In [33]:
from lionagi.protocols.generic.pile import Pile

msgs_ = Pile.from_dict(msgs)

In [34]:
len(msgs_)

4

In [35]:
msgs_

Pile(4)

In [36]:
g

Instruction(id=IDType(38f31b61-3442-4598-94b0-1e1f7b28656e), created_at=1736806393.238253, metadata={}, content={'context': [], 'instruction': 'hello? you should test the test_func with a sample queries, if already did, just test again', 'tool_schemas': {'tools': [{'type': 'function', 'function': {'name': 'test_func', 'description': 'this is a test function it will return the query as it is', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'str: query to be returned'}}, 'required': ['query']}}}]}, 'respond_schema_info': {'$defs': {'ActionRequestModel': {'properties': {'function': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': "Name of the function to call from the provided `tool_schemas`. If no `tool_schemas` exist, set to None or leave blank. Never invent new function names outside what's given.", 'examples': ['multiply', 'create_user'], 'title': 'Function'}, 'arguments': {'anyOf': [{'type': 'object'}, {'typ

In [37]:
for i in msgs_:
    print(i.class_name())

Instruction
AssistantResponse
ActionRequest
ActionResponse


In [38]:
french.chat_model.to_dict()

{'provider': 'groq',
 'endpoint': {'provider': 'groq',
  'base_url': 'https://api.groq.com/openai/v1',
  'endpoint': 'chat/completions',
  'endpoint_params': None,
  'method': 'post',
  'openai_compatible': True,
  'required_kwargs': {'messages', 'model'},
  'optional_kwargs': {'audio',
   'frequency_penalty',
   'max_completion_tokens',
   'metadata',
   'modalities',
   'parallel_tool_calls',
   'prediction',
   'presence_penalty',
   'reasoning_effort',
   'seed',
   'service_tier',
   'stop',
   'store',
   'stream',
   'stream_options',
   'temperature',
   'tool_choice',
   'tools',
   'top_p',
   'user'},
  'deprecated_kwargs': {'function_call', 'functions', 'max_tokens'},
  'is_invokeable': True,
  'is_streamable': True,
  'requires_tokens': True,
  'api_version': None,
  'allowed_roles': None},
 'api_key': 'GROQ_API_KEY',
 'processor_config': {'queue_capacity': 100,
  'capacity_refresh_time': 60,
  'interval': None,
  'limit_requests': None,
  'limit_tokens': None},
 'invoke_w

In [39]:
g = Branch.from_dict(d_)

In [40]:
g.logs.to_df()

,id,created_at,metadata,content
0,7801c9d8-e2c4-4777-bbee-5447c4717727,2025-01-13 17:13:14.971775,{'lion_class': 'lionagi.protocols.generic.log....,"{'id': 'e11a895f-f695-43c7-9604-96963edf86de',..."
1,4f7d39f1-e0ea-4b22-a6d0-a214eca7ab4f,2025-01-13 17:13:14.975282,{'lion_class': 'lionagi.protocols.generic.log....,"{'id': 'c96d328d-5730-4585-9d38-19692e939060',..."


In [41]:
g.chat_model.to_dict()

{'provider': 'groq',
 'endpoint': {'provider': 'groq',
  'base_url': 'https://api.groq.com/openai/v1',
  'endpoint': 'chat/completions',
  'endpoint_params': None,
  'method': 'post',
  'openai_compatible': True,
  'required_kwargs': {'messages', 'model'},
  'optional_kwargs': {'audio',
   'frequency_penalty',
   'max_completion_tokens',
   'metadata',
   'modalities',
   'parallel_tool_calls',
   'prediction',
   'presence_penalty',
   'reasoning_effort',
   'seed',
   'service_tier',
   'stop',
   'store',
   'stream',
   'stream_options',
   'temperature',
   'tool_choice',
   'tools',
   'top_p',
   'user'},
  'deprecated_kwargs': {'function_call', 'functions', 'max_tokens'},
  'is_invokeable': True,
  'is_streamable': True,
  'requires_tokens': True,
  'api_version': None,
  'allowed_roles': None},
 'api_key': 'GROQ_API_KEY',
 'processor_config': {'queue_capacity': 100,
  'capacity_refresh_time': 60,
  'interval': None,
  'limit_requests': None,
  'limit_tokens': None},
 'invoke_w

In [42]:
from lionagi import ops_types

instruct = ops_types.Instruct(
    instruction="hello? you should test the test_func with a sample queries, if already did, just test again",
    actions=True,
    reason=True,
)

result = await g.operate(
    instruct=instruct,
    response_format=Test,
    return_operative=True,
    tools=test_func,
    # tools=test_func,
    # invoke_actions=False,
)

In [43]:
g.to_df()

,created_at,role,content,id,sender,recipient,metadata
0,2025-01-13 17:13:13.238253,user,"{'context': [], 'instruction': 'hello? you sho...",38f31b61-3442-4598-94b0-1e1f7b28656e,user,9acac2e6-eb6a-422e-bcd2-337f8ccdca08,{'lion_class': 'lionagi.protocols.messages.ins...
1,2025-01-13 17:13:14.971864,assistant,"{'assistant_response': '```json { ""comments""...",5a50c84f-e00e-45f2-a37a-84162a00a7bb,9acac2e6-eb6a-422e-bcd2-337f8ccdca08,user,{'model_response': {'id': 'chatcmpl-6e75b868-6...
2,2025-01-13 17:13:14.975374,action,"{'action_request': {'function': 'test_func', '...",9aa90c37-326f-45b5-8840-deba2267c674,9acac2e6-eb6a-422e-bcd2-337f8ccdca08,3376c9a4-5f30-4604-b7e5-2240fa0f14a2,{'lion_class': 'lionagi.protocols.messages.act...
3,2025-01-13 17:13:14.975498,action,{'action_request_id': '9aa90c37-326f-45b5-8840...,6202fb61-7448-4798-a651-b4e38a5f9eed,3376c9a4-5f30-4604-b7e5-2240fa0f14a2,9acac2e6-eb6a-422e-bcd2-337f8ccdca08,{'lion_class': 'lionagi.protocols.messages.act...
4,2025-01-13 17:13:15.131952,user,{'context': [{'action_request_id': '9aa90c37-3...,90d47086-4d62-4e81-96c4-e450f3a0f3a5,user,9acac2e6-eb6a-422e-bcd2-337f8ccdca08,{'lion_class': 'lionagi.protocols.messages.ins...
5,2025-01-13 17:13:15.892710,assistant,"{'assistant_response': '```json { ""comments""...",bdf2b7a5-fff3-42fb-8b73-f26b4f1be4c1,9acac2e6-eb6a-422e-bcd2-337f8ccdca08,user,{'model_response': {'id': 'chatcmpl-18b6e8c8-0...
6,2025-01-13 17:13:15.893625,action,"{'action_request': {'function': 'test_func', '...",407f8f42-f978-45e6-a61f-38a27119815c,9acac2e6-eb6a-422e-bcd2-337f8ccdca08,24e1f19e-3b14-4918-8aa6-90b7d3a02013,{'lion_class': 'lionagi.protocols.messages.act...
7,2025-01-13 17:13:15.893753,action,{'action_request_id': '407f8f42-f978-45e6-a61f...,3f16d103-fb12-4e0a-8c44-401b83f7fe5b,24e1f19e-3b14-4918-8aa6-90b7d3a02013,9acac2e6-eb6a-422e-bcd2-337f8ccdca08,{'lion_class': 'lionagi.protocols.messages.act...
